I have hourly data for geographical positions, speed over goround, and CO2 emissions from 100 unique ships for the year 2019. I have also loaded a list of major ports across the world. Assuming the sample is complete and representative, I intened to present a simple analysis of shipping emissions.

In [1]:
import pandas as pd
import sqlite3
import geopy

The first step is loading the data!

In [4]:
con = sqlite3.connect("dwh/db")

ships = pd.read_sql_query('select * from ships', con)
ports = pd.read_sql_query('select * from ports', con)
ships.info()
ports.info()
con.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876000 entries, 0 to 875999
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   index      876000 non-null  int64  
 1   IMO        876000 non-null  int64  
 2   TIME       876000 non-null  object 
 3   SOG        876000 non-null  float64
 4   LONGITUDE  876000 non-null  float64
 5   LATITUDE   876000 non-null  float64
 6   E_CO2_kg   876000 non-null  float64
dtypes: float64(4), int64(2), object(1)
memory usage: 46.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     177 non-null    int64  
 1   OBJECTID  177 non-null    int64  
 2   Postal    177 non-null    object 
 3   Lat       175 non-null    float64
 4   Lng       175 non-null    float64
 5   Type      177 non-null    object 
 6   ISO3      177 non-null    object 
 7 

I defined a function to calculate the distance between two coordinates using the geodesic function from geopy.


In [5]:
from geopy.distance import geodesic

def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

I proceded with running a code to use the distance function with two for loops to calculate distances between every observation and every port. 

I defined a **rectangular** **space** with 0.2 degrees LAT and 0.3 degrees LONG on each side and the ship's observed AIS position in the middle. 0.3 degrees Longitude is roughly 12 nautical miles(UNCLOS definition of territorial waters) at 50 degrees North which is an average for European ports. 0.2 degrees lattitude amounts to a similar distance at 50 degrees North.

In [6]:

ships['ports_in_range_indexes'] = None

for index, row in ships.iterrows():
    lng_distance = 0.3
    lat_distance = 0.2
    condition1 = ports['Lng'] > row['LONGITUDE'] - lng_distance
    condition2 = ports['Lng'] < row['LONGITUDE'] + lng_distance
    condition3 = ports['Lat'] > row['LATITUDE'] - lat_distance
    condition4 = ports['Lat'] < row['LATITUDE'] + lat_distance

    result_indexes = ports[condition1 & condition2 & condition3 & condition4].index
    if result_indexes.empty:
        continue
    else:
        ships.at[index, 'ports_in_range_indexes'] = result_indexes.tolist()


In [7]:
ships_box.info()

NameError: name 'ships_box' is not defined

Next, I filter the observations where ships where close to ports.

In [8]:
ships_around_ports = ships[ships['ports_in_range_indexes'].notna()]
ships_around_ports.to_sql

In [9]:
ships_around_ports

,index,IMO,TIME,SOG,LONGITUDE,LATITUDE,E_CO2_kg,ports_in_range_indexes
18,18,9783526,2019-11-19 02:00:00,16.929167,-5.876352,35.913180,16680.828581,[81]
89,89,9783461,2019-05-26 03:00:00,8.650000,32.336667,31.198333,1396.629000,[108]
365,365,9619971,2019-03-16 03:00:00,9.366667,32.385517,31.364083,4441.347806,[108]
401,401,9632155,2019-03-30 10:00:00,0.015789,-5.568333,35.876667,1517.140800,[81]
503,503,9619921,2019-01-12 14:00:00,9.500000,32.310867,31.108750,4660.177824,[108]
...,...,...,...,...,...,...,...,...
875496,875496,9783514,2019-02-10 02:00:00,6.150000,32.382088,31.353047,1396.629000,[108]
875589,875589,9769271,2019-12-04 18:00:00,0.000000,32.341667,31.455000,1925.230500,[108]
875778,875778,9619921,2019-05-07 00:00:00,0.100000,32.360050,31.463217,1851.117300,[108]
875795,875795,9776200,2019-09-03 20:00:00,0.200000,32.320008,31.461775,1925.230500,[108]


I can now use the distance function defined earlier on and run a code with for loops applied on a smaller dataset. 

In [15]:
matches = []

for index, ship in ships_around_ports.iterrows():
    min_distance = float('inf') 
    closest_port_index = None

    for value in ship['ports_in_range_indexes']:
        lat_port = float(ports.loc[value, 'Lat'])
        lon_port = float(ports.loc[value, 'Lng'])

        distance = calculate_distance(ships['LATITUDE'], ships['LONGITUDE'], lat_port, lon_port)

        if distance < min_distance:
            min_distance = distance
            closest_port_index = value

    ships_around_ports.at[index, 'closest_port'] = ports.loc[closest_port_index, 'PORT_NAME']
    ships_around_ports.at[index, 'continent'] = ports.loc[closest_port_index, 'Continent']
    ships_around_ports.at[index, 'distance_to_closest_port'] = min_distance
        

ships.to_csv('final_ship.csv', index=False)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().